# Import used libraries

In [9]:
#System libraries
import numpy as np
import random
import json
import pickle

#preprocessing library 
import nltk
from nltk.stem.lancaster import LancasterStemmer

#model libraries 
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold


#GUI Libraries
import tkinter as tk
from tkinter import scrolledtext
from tkinter import ttk  
import warnings

In [10]:
# used for preprocessing 
nltk.download('punkt')
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Khater\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# import json dataset
with open('intents2 edit2.json') as json_data:
    intents = json.load(json_data)

In [12]:
# Preprocess the data
words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

332 documents
60 classes ['Anger Management', 'Bipolar Disorder', 'COVID-19', 'Dengue', 'Ebola', 'Grief and Loss', 'Hyperglycaemia', 'Hypoglycaemia', 'Obsessive-Compulsive Disorder (OCD)', 'Personality Disorders', 'Phobias', 'Post-Traumatic Stress Disorder (PTSD)', 'Relationship Issues', 'Schizophrenia', 'Sleep Issues', 'Stress Management', 'Substance Abuse', 'Trauma', 'alcohol-related liver disease', 'allergy', 'anal cancer', 'anxiety', 'appendicitis', 'asthma', 'bladder cancer', 'bladder problems', 'breast problems', 'chickenpox', 'common cold', 'dehydration', 'depression', 'diabetic retinopathy', 'diarrhoea', 'eating or weight problems', 'emotional problems', 'eye cancer', 'feedback', 'fibromyalgia', 'glandular fever', 'goodbye', 'greeting', 'hay fever', 'headache problems', 'help', 'hepatitis_B', 'influenza', 'jaundice', 'kidney cancer', 'lung problems', 'mental_health', 'name', 'negation', 'positive_feelings', 'problem_solving', 'reproductive health problems', 'self-esteem', 'stro

In [27]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])




In [ ]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) 
            
pickle.dump(train_x, open('train_x.pkl', 'wb')) 
pickle.dump(train_y, open('train_y.pkl', 'wb'))

# Words and documents
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(documents, open('documents.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# buliding Model 

In [18]:
def create_model():
    tf.keras.backend.clear_session()
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(len(train_x[0]),)))
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(len(train_y[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Perform cross-validation
num_folds = 2
kf = KFold(n_splits=num_folds, shuffle=True)



for train_index, test_index in kf.split(train_x):
    train_x_fold, test_x_fold = np.array(train_x)[train_index], np.array(train_x)[test_index]
    train_y_fold, test_y_fold = np.array(train_y)[train_index], np.array(train_y)[test_index]

    model = create_model()
    model.fit(train_x_fold, train_y_fold, epochs=500, batch_size=8, verbose=1)

#model.save("Medical-chatbot.h5")

Epoch 1/500
21/21 [==============================] - 1s 5ms/step - loss: 4.0940 - accuracy: 0.0120
Epoch 2/500
21/21 [==============================] - 0s 4ms/step - loss: 3.9062 - accuracy: 0.1325
Epoch 3/500
21/21 [==============================] - 0s 5ms/step - loss: 3.4588 - accuracy: 0.1386
Epoch 4/500
21/21 [==============================] - 0s 5ms/step - loss: 2.8181 - accuracy: 0.2831
Epoch 5/500
21/21 [==============================] - 0s 5ms/step - loss: 2.1320 - accuracy: 0.5422
Epoch 6/500
21/21 [==============================] - 0s 5ms/step - loss: 1.4560 - accuracy: 0.7048
Epoch 7/500
21/21 [==============================] - 0s 5ms/step - loss: 1.0009 - accuracy: 0.7651
Epoch 8/500
21/21 [==============================] - 0s 5ms/step - loss: 0.7109 - accuracy: 0.8193
Epoch 9/500
21/21 [==============================] - 0s 4ms/step - loss: 0.5282 - accuracy: 0.8434
Epoch 10/500
21/21 [==============================] - 0s 5ms/step - loss: 0.4082 - accuracy: 0.8614
Epoch 11/

21/21 [==============================] - 0s 4ms/step - loss: 0.2245 - accuracy: 0.8494
Epoch 84/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2209 - accuracy: 0.8554
Epoch 85/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2154 - accuracy: 0.8554
Epoch 86/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2087 - accuracy: 0.8494
Epoch 87/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2091 - accuracy: 0.8614
Epoch 88/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2076 - accuracy: 0.8434
Epoch 89/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2077 - accuracy: 0.8795
Epoch 90/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2046 - accuracy: 0.8614
Epoch 91/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2147 - accuracy: 0.8554
Epoch 92/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2067 - accuracy: 0.8675
Epoch 93/500


21/21 [==============================] - 0s 4ms/step - loss: 0.1920 - accuracy: 0.8373
Epoch 165/500
21/21 [==============================] - 0s 5ms/step - loss: 0.1933 - accuracy: 0.8614
Epoch 166/500
21/21 [==============================] - 0s 5ms/step - loss: 0.1965 - accuracy: 0.8494
Epoch 167/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1932 - accuracy: 0.8675
Epoch 168/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1964 - accuracy: 0.8554
Epoch 169/500
21/21 [==============================] - 0s 5ms/step - loss: 0.1963 - accuracy: 0.8795
Epoch 170/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1954 - accuracy: 0.8554
Epoch 171/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1940 - accuracy: 0.8554
Epoch 172/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1941 - accuracy: 0.8795
Epoch 173/500
21/21 [==============================] - 0s 5ms/step - loss: 0.1936 - accuracy: 0.8614
Epoc

21/21 [==============================] - 0s 4ms/step - loss: 0.1914 - accuracy: 0.8614
Epoch 246/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1903 - accuracy: 0.8554
Epoch 247/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1901 - accuracy: 0.8675
Epoch 248/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1894 - accuracy: 0.8554
Epoch 249/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2375 - accuracy: 0.8554
Epoch 250/500
21/21 [==============================] - 0s 4ms/step - loss: 1.8497 - accuracy: 0.6265
Epoch 251/500
21/21 [==============================] - 0s 4ms/step - loss: 1.0691 - accuracy: 0.6747
Epoch 252/500
21/21 [==============================] - 0s 4ms/step - loss: 0.4403 - accuracy: 0.8133
Epoch 253/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2524 - accuracy: 0.8675
Epoch 254/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2100 - accuracy: 0.8675
Epoc

21/21 [==============================] - 0s 4ms/step - loss: 0.1839 - accuracy: 0.8795
Epoch 327/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1850 - accuracy: 0.8675
Epoch 328/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1877 - accuracy: 0.8434
Epoch 329/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1903 - accuracy: 0.8554
Epoch 330/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1859 - accuracy: 0.8554
Epoch 331/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1845 - accuracy: 0.8614
Epoch 332/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1872 - accuracy: 0.8494
Epoch 333/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1847 - accuracy: 0.8735
Epoch 334/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1837 - accuracy: 0.8614
Epoch 335/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1887 - accuracy: 0.8614
Epoc

...

21/21 [==============================] - 0s 4ms/step - loss: 0.1042 - accuracy: 0.9157
Epoch 152/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1039 - accuracy: 0.9277
Epoch 153/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0995 - accuracy: 0.9217
Epoch 154/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1035 - accuracy: 0.9217
Epoch 155/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1069 - accuracy: 0.9217
Epoch 156/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1003 - accuracy: 0.9217
Epoch 157/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1063 - accuracy: 0.9217
Epoch 158/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0997 - accuracy: 0.9217
Epoch 159/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1061 - accuracy: 0.9096
Epoch 160/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1072 - accuracy: 0.9157
Epoc

21/21 [==============================] - 0s 4ms/step - loss: 0.1007 - accuracy: 0.9036
Epoch 233/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0993 - accuracy: 0.9217
Epoch 234/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1005 - accuracy: 0.9096
Epoch 235/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0980 - accuracy: 0.9217
Epoch 236/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1003 - accuracy: 0.9217
Epoch 237/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1001 - accuracy: 0.9277
Epoch 238/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0985 - accuracy: 0.9277
Epoch 239/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0981 - accuracy: 0.9217
Epoch 240/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0998 - accuracy: 0.9157
Epoch 241/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0999 - accuracy: 0.9157
Epoc

21/21 [==============================] - 0s 4ms/step - loss: 0.0965 - accuracy: 0.9036
Epoch 314/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0956 - accuracy: 0.9337
Epoch 315/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0988 - accuracy: 0.9217
Epoch 316/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0957 - accuracy: 0.9157
Epoch 317/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0980 - accuracy: 0.9217
Epoch 318/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0989 - accuracy: 0.9157
Epoch 319/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0952 - accuracy: 0.9277
Epoch 320/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0999 - accuracy: 0.9157
Epoch 321/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0953 - accuracy: 0.9157
Epoch 322/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0956 - accuracy: 0.9217
Epoc

21/21 [==============================] - 0s 4ms/step - loss: 0.0968 - accuracy: 0.9217
Epoch 395/500
21/21 [==============================] - 0s 5ms/step - loss: 0.0949 - accuracy: 0.9096
Epoch 396/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0959 - accuracy: 0.9096
Epoch 397/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0960 - accuracy: 0.9337
Epoch 398/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0962 - accuracy: 0.9157
Epoch 399/500
21/21 [==============================] - 0s 3ms/step - loss: 0.0943 - accuracy: 0.9217
Epoch 400/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0952 - accuracy: 0.9217
Epoch 401/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0963 - accuracy: 0.9277
Epoch 402/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0995 - accuracy: 0.9157
Epoch 403/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0967 - accuracy: 0.9217
Epoc

21/21 [==============================] - 0s 4ms/step - loss: 0.0938 - accuracy: 0.9217
Epoch 476/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0947 - accuracy: 0.9157
Epoch 477/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0947 - accuracy: 0.9157
Epoch 478/500
21/21 [==============================] - 0s 4ms/step - loss: 0.0946 - accuracy: 0.9157
Epoch 479/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1185 - accuracy: 0.9217
Epoch 480/500
21/21 [==============================] - 0s 4ms/step - loss: 0.1714 - accuracy: 0.8976
Epoch 481/500
21/21 [==============================] - 0s 4ms/step - loss: 1.0089 - accuracy: 0.7771
Epoch 482/500
21/21 [==============================] - 0s 4ms/step - loss: 0.7724 - accuracy: 0.8012
Epoch 483/500
21/21 [==============================] - 0s 4ms/step - loss: 0.3321 - accuracy: 0.8675
Epoch 484/500
21/21 [==============================] - 0s 4ms/step - loss: 0.2752 - accuracy: 0.9036
Epoc

In [25]:
# Load the trained model
model = keras.models.load_model('chatbot medical2.h5')

# Create the chatbot interface
def classify_and_respond(event=None):
    user_input = input_text.get("1.0", "end-1c")
    input_text.delete("1.0", tk.END)
    if user_input.lower() == 'quit':
        root.quit()
        return
    results = classify(user_input)
    if results:
        for intent in intents['intents']:
            if intent['tag'] == results[0][0]:
                response = np.random.choice(intent['responses'])
                update_chat("You: " + user_input, "ChatBot: " + response, "response")
    else:
        update_chat("You: " + user_input, "ChatBot: I'm sorry, but I don't understand. Can you please rephrase?", "error")

def classify(sentence):
    # Generate probabilities from the model
    results = model.predict(np.array([bow(sentence, words)]))[0]
    # Filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results) if r > 0.25]
    # Sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def clean_up_sentence(sentence):
    # Tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # Stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    # Tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # Bag of words
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s" % w)
    return np.array(bag)

def update_chat(user_message, bot_message, tag):
    response_text.insert(tk.END, user_message + "\n", tag)
    response_text.insert(tk.END, bot_message + "\n", tag)
    response_text.see(tk.END)


# GUI

In [26]:
root = tk.Tk()
root.title("Medical ChatBot")
root.geometry("700x700")  # Adjust the window size

# Set the background color
root.configure(bg="lightblue")

# Create input and response text areas
input_label = tk.Label(root, text="Enter your message :", bg="lightblue", fg="darkblue", font=("Bold", 12))
input_label.pack(pady=8)
input_text = tk.Text(root, height=1, width=50, bg="white")
input_text.pack(pady=8, padx=6)  # Added padding for better spacing
input_text.bind("<Return>", classify_and_respond) 

response_label = tk.Label(root, text="ChatBot Conversation", bg="lightblue", fg="darkblue", font=("Bold", 12))
response_label.pack(pady=8)
response_text = scrolledtext.ScrolledText(root, height=20, width=100, bg="white")
response_text.pack(pady=10, padx=10)  # Added padding for better spacing

# Create enhanced styled buttons
style = ttk.Style()
style.configure("TButton", foreground="black", background="blue", font=("Bold", 15))

send_button = ttk.Button(root, text="Send", command=classify_and_respond, style="TButton")
send_button.pack(pady=15)


root.mainloop()

1/1 [==============================] - 0s 31ms/step
